In [ ]:
# 🐕 Dog Emotion Classification - NFNet Cross-Validation Training

Notebook này sẽ:
1. **Clone repository** từ GitHub và cài đặt dependencies
2. **Import NFNet module** từ `dog_emotion_classification.nfnet`
3. **Download dataset** dog emotion classification  
4. **Train NFNet** với 50 epochs sử dụng K-Fold Cross Validation
5. **Evaluate** với cross-validation scores và confusion matrix
6. **Download models** và results về máy

---
**Author**: Dog Emotion Research Team  
**Date**: 2025  
**Runtime**: Google Colab (GPU T4/V100 recommended)  
**Training**: NFNet (Normalizer-Free Network) với Cross Validation  
**Repository**: https://github.com/hoangh-e/dog-emotion-recognition-hybrid.git  
**Module**: `dog_emotion_classification.nfnet`


In [ ]:
# 🔧 STEP 1: Clone Repository và Setup Environment
import os
import sys

# Clone repository từ GitHub
REPO_URL = "https://github.com/hoangh-e/dog-emotion-recognition-hybrid.git"
REPO_NAME = "dog-emotion-recognition-hybrid"

if not os.path.exists(REPO_NAME):
    print(f"📥 Cloning repository from {REPO_URL}")
    !git clone {REPO_URL}
    print("✅ Repository cloned successfully!")
else:
    print(f"✅ Repository already exists: {REPO_NAME}")

# Change to repository directory
os.chdir(REPO_NAME)
print(f"📁 Current directory: {os.getcwd()}")

# Add to Python path để import modules
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())
    print("✅ Added repository to Python path")

# Install required packages
print("📦 Installing dependencies...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install opencv-python-headless pillow pandas tqdm gdown albumentations matplotlib seaborn
!pip install scikit-learn timm ultralytics

print("✅ Environment setup complete!")


In [ ]:
# 🧪 STEP 2: Import Libraries và NFNet Module
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import json
import zipfile
import warnings
warnings.filterwarnings('ignore')

# 🎯 Import NFNet từ custom module
try:
    from dog_emotion_classification.nfnet import (
        load_nfnet_model, 
        predict_emotion_nfnet,
        get_nfnet_transforms,
        create_nfnet_model,
        NFNetModel
    )
    print("✅ Successfully imported NFNet module from dog_emotion_classification.nfnet")
    print("📋 Available functions:")
    print("   - load_nfnet_model()")
    print("   - predict_emotion_nfnet()")
    print("   - get_nfnet_transforms()")
    print("   - create_nfnet_model()")
    print("   - NFNetModel class")
except ImportError as e:
    print(f"❌ Failed to import NFNet module: {e}")
    print("Please ensure you're in the repository directory and the module exists.")
    raise

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🔥 PyTorch version: {torch.__version__}")
print(f"🚀 CUDA available: {torch.cuda.is_available()}")
print(f"🎯 Using device: {device}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)


In [ ]:
# Dataset and Training
class DogEmotionDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.classes = ['angry', 'happy', 'relaxed', 'sad']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        self.samples = []
        for class_name in self.classes:
            class_dir = self.data_dir / class_name
            if class_dir.exists():
                for img_path in class_dir.glob('*.jpg'):
                    self.samples.append((str(img_path), self.class_to_idx[class_name]))
        
        print(f"📁 Dataset: {len(self.samples)} images, Classes: {self.classes}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        try:
            image = Image.open(img_path).convert('RGB').resize((224, 224), Image.LANCZOS)
            if self.transform:
                image = self.transform(image)
            return image, label
        except:
            dummy = Image.new('RGB', (224, 224), 'black')
            if self.transform:
                dummy = self.transform(dummy)
            return dummy, label

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Config
EPOCHS, BATCH_SIZE, LR = 50, 16, 1e-4
NUM_CLASSES, K_FOLDS = 4, 5
EMOTION_CLASSES = ['angry', 'happy', 'relaxed', 'sad']

# Dataset
dataset = DogEmotionDataset("dog_emotion_dataset", train_transform)
labels = [sample[1] for sample in dataset.samples]
kfold = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

print(f"📊 Config: {EPOCHS} epochs, batch {BATCH_SIZE}, lr {LR}, device {device}")

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for data, target in tqdm(loader, desc="Training"):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, pred = torch.max(output, 1)
        total += target.size(0)
        correct += (pred == target).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    all_preds, all_targets = [], []
    
    with torch.no_grad():
        for data, target in tqdm(loader, desc="Validation"):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            
            total_loss += loss.item()
            _, pred = torch.max(output, 1)
            total += target.size(0)
            correct += (pred == target).sum().item()
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    return total_loss / len(loader), 100. * correct / total, all_preds, all_targets

print("✅ Dataset and functions ready!")


In [ ]:
# Cross-Validation Training
print("🎯 Starting 5-Fold Cross-Validation Training...")
fold_results, all_val_accs = [], []

for fold, (train_idx, val_idx) in enumerate(kfold.split(range(len(dataset)), labels)):
    print(f"\n{'='*60}\n🔄 FOLD {fold + 1}/5\n{'='*60}")
    
    # Data loaders
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=2)
    val_dataset = DogEmotionDataset("dog_emotion_dataset", val_transform)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler, num_workers=2)
    
    print(f"📊 Training: {len(train_idx)}, Validation: {len(val_idx)}")
    
    # Model
    model = NFNetModel(num_classes=NUM_CLASSES, variant='F0').to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    # Training history
    train_losses, train_accs, val_losses, val_accs = [], [], [], []
    best_val_acc, best_model_state = 0.0, None
    
    # Training loop
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch + 1}/{EPOCHS}")
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, val_preds, val_targets = validate_epoch(model, val_loader, criterion, device)
        scheduler.step()
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        print(f"Train: {train_loss:.4f}, {train_acc:.2f}% | Val: {val_loss:.4f}, {val_acc:.2f}%")
    
    # Final evaluation
    model.load_state_dict(best_model_state)
    final_loss, final_acc, final_preds, final_targets = validate_epoch(model, val_loader, criterion, device)
    
    print(f"\n✅ Fold {fold + 1} completed! Best Val Acc: {best_val_acc:.2f}%")
    
    fold_results.append({
        'fold': fold + 1,
        'train_losses': train_losses,
        'train_accs': train_accs,
        'val_losses': val_losses,
        'val_accs': val_accs,
        'best_val_acc': best_val_acc,
        'final_preds': final_preds,
        'final_targets': final_targets,
        'model_state': best_model_state
    })
    all_val_accs.append(best_val_acc)
    torch.cuda.empty_cache()

mean_acc = np.mean(all_val_accs)
std_acc = np.std(all_val_accs)

print(f"\n🎉 CROSS-VALIDATION COMPLETED!")
print(f"{'='*60}")
print(f"📊 Mean Accuracy: {mean_acc:.2f}% ± {std_acc:.2f}%")
print(f"📈 Fold Accuracies: {[f'{acc:.2f}%' for acc in all_val_accs]}")
print(f"🏆 Best Fold: {np.argmax(all_val_accs) + 1} ({max(all_val_accs):.2f}%)")


In [ ]:
# Visualization and Results
plt.style.use('default')
sns.set_palette("husl")
fig = plt.figure(figsize=(20, 15))

# Training curves
ax1 = plt.subplot(2, 3, 1)
for result in fold_results:
    epochs = range(1, len(result['train_losses']) + 1)
    plt.plot(epochs, result['train_losses'], label=f"Fold {result['fold']}", alpha=0.7)
plt.title('Training Loss Across Folds', fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Validation curves
ax2 = plt.subplot(2, 3, 2)
for result in fold_results:
    epochs = range(1, len(result['val_accs']) + 1)
    plt.plot(epochs, result['val_accs'], label=f"Fold {result['fold']}", alpha=0.7)
plt.title('Validation Accuracy Across Folds', fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True, alpha=0.3)

# Fold comparison
ax3 = plt.subplot(2, 3, 3)
fold_names = [f'Fold {i+1}' for i in range(K_FOLDS)]
bars = plt.bar(fold_names, all_val_accs, color=sns.color_palette("husl", K_FOLDS))
plt.title('Best Validation Accuracy by Fold', fontweight='bold')
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)

for bar, acc in zip(bars, all_val_accs):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{acc:.1f}%', ha='center', va='bottom', fontweight='bold')
plt.axhline(y=mean_acc, color='red', linestyle='--', alpha=0.7, label=f'Mean: {mean_acc:.1f}%')
plt.legend()

# Confusion Matrix
best_fold_idx = np.argmax(all_val_accs)
best_result = fold_results[best_fold_idx]
ax4 = plt.subplot(2, 3, 4)
cm = confusion_matrix(best_result['final_targets'], best_result['final_preds'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=EMOTION_CLASSES, yticklabels=EMOTION_CLASSES)
plt.title(f'Confusion Matrix - Best Fold ({best_fold_idx+1})', fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Training vs Validation
ax5 = plt.subplot(2, 3, 5)
epochs = range(1, len(best_result['train_accs']) + 1)
plt.plot(epochs, best_result['train_accs'], label='Training', linewidth=2)
plt.plot(epochs, best_result['val_accs'], label='Validation', linewidth=2)
plt.title(f'Training vs Validation - Best Fold ({best_fold_idx+1})', fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True, alpha=0.3)

# Statistics
ax6 = plt.subplot(2, 3, 6)
ax6.axis('off')
stats_text = f"""
📊 NFNet Cross-Validation Results

🎯 Model Performance:
   • Mean Accuracy: {mean_acc:.2f}% ± {std_acc:.2f}%
   • Best Fold: {best_fold_idx+1} ({max(all_val_accs):.2f}%)
   • Worst Fold: {np.argmin(all_val_accs)+1} ({min(all_val_accs):.2f}%)

⚙️ Training Configuration:
   • Architecture: NFNet-F0 (Normalizer-Free)
   • Epochs per fold: {EPOCHS}
   • Batch size: {BATCH_SIZE}
   • Learning rate: {LR}
   • Device: {device}

📈 Data Information:
   • Total samples: {len(dataset)}
   • Classes: {len(EMOTION_CLASSES)}
   • Folds: {K_FOLDS} (stratified)
"""

ax6.text(0.1, 0.9, stats_text, transform=ax6.transAxes, fontsize=12,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle="round,pad=0.5", facecolor="lightcyan", alpha=0.8))

plt.tight_layout()
plt.savefig('nfnet_cross_validation_results.png', dpi=300, bbox_inches='tight')
plt.show()

# Save model
best_model = NFNetModel(num_classes=NUM_CLASSES, variant='F0')
best_model.load_state_dict(fold_results[best_fold_idx]['model_state'])
model_filename = f'nfnet_best_fold_{best_fold_idx+1}_acc_{max(all_val_accs):.2f}.pth'

torch.save({
    'model_state_dict': best_model.state_dict(),
    'model_config': {'num_classes': NUM_CLASSES, 'architecture': 'NFNet-F0', 'variant': 'F0'},
    'training_info': {
        'best_fold': best_fold_idx + 1,
        'best_accuracy': max(all_val_accs),
        'mean_accuracy': mean_acc,
        'std_accuracy': std_acc,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LR
    },
    'class_names': EMOTION_CLASSES
}, model_filename)

# Save results
results_filename = 'nfnet_training_results.json'
with open(results_filename, 'w') as f:
    json.dump({
        'cross_validation_results': {
            'mean_accuracy': float(mean_acc),
            'std_accuracy': float(std_acc),
            'fold_accuracies': [float(acc) for acc in all_val_accs],
            'best_fold': int(best_fold_idx + 1),
            'best_accuracy': float(max(all_val_accs))
        },
        'training_config': {'epochs': EPOCHS, 'batch_size': BATCH_SIZE, 'learning_rate': LR, 'device': str(device)},
        'dataset_info': {'total_samples': len(dataset), 'num_classes': NUM_CLASSES, 'class_names': EMOTION_CLASSES}
    }, f, indent=2)

# Download files
try:
    from google.colab import files
    print("\n📥 Downloading files...")
    files.download(model_filename)
    files.download(results_filename)
    files.download('nfnet_cross_validation_results.png')
    print("✅ Files downloaded!")
except ImportError:
    print("📁 Files saved locally")

print(f"\n🎉 NFNET TRAINING COMPLETED!")
print(f"{'='*80}")
print(f"📊 FINAL RESULTS:")
print(f"   🎯 Mean Accuracy: {mean_acc:.2f}% ± {std_acc:.2f}%")
print(f"   🏆 Best Fold: {best_fold_idx+1} with {max(all_val_accs):.2f}% accuracy")
print(f"   📈 All Fold Accuracies: {[f'{acc:.2f}%' for acc in all_val_accs]}")
print(f"   💾 Model: {model_filename}")
print(f"   📊 Results: {results_filename}")
print(f"{'='*80}")

print(f"\n📋 HOW TO USE:")
print(f"model = NFNetModel(num_classes=4, variant='F0')")
print(f"checkpoint = torch.load('{model_filename}')")
print(f"model.load_state_dict(checkpoint['model_state_dict'])")
print(f"Classes: {EMOTION_CLASSES}")

print(f"\n🎯 NFNet training completed successfully!")


In [ ]:
#!/usr/bin/env python3
"""
🤖 NFNet Cross-Validation Training for Dog Emotion Recognition
===============================================================

Complete pipeline for training NFNet (Normalizer-Free Networks) on dog emotion dataset with:
- Automatic dataset download and preparation
- 5-fold stratified cross-validation
- 50 epochs training per fold
- Comprehensive visualization and evaluation
- Model saving and download

Author: Dog Emotion Recognition Team
Date: 2025
"""

import os
import warnings
import time
import zipfile
import json

# Suppress warnings
warnings.filterwarnings('ignore')

print("🚀 Starting NFNet Cross-Validation Training Pipeline")
print("=" * 60)

# Install packages
packages = [
    'torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121',
    'gdown scikit-learn matplotlib seaborn Pillow numpy pandas tqdm opencv-python'
]

for package in packages:
    print(f"Installing {package.split()[0]}...")
    os.system(f"pip install {package} -q")

print("✅ All packages installed successfully!")

# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import gdown
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import random

# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Download dataset
dataset_url = "https://drive.google.com/uc?id=1ZAgz5u64i3LDbwMFpBXjzsKt6FrhNGdW"
dataset_zip = "dog_emotion_dataset.zip"

if not os.path.exists(dataset_zip):
    print("📥 Downloading dataset...")
    gdown.download(dataset_url, dataset_zip, quiet=False)
    print("✅ Dataset downloaded successfully!")

# Extract dataset
dataset_dir = "dog_emotion_dataset"
if not os.path.exists(dataset_dir):
    print("📂 Extracting dataset...")
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall()
    print("✅ Dataset extracted successfully!")

print("✅ Environment setup complete!")
